In [1]:
import os

# Subir un nivel respecto a la carpeta actual
os.chdir("..")

In [2]:
from Utils import load_parameters_file, from_matrix_to_preset, MSE, denormalize_preset, normalize_preset, pretty_print, PARAM_NAMES, from_preset_to_matrix
from Synth import Synth
import numpy as np
import cma
import matplotlib.pyplot as plt
from multiprocessing import Pool
from parallelEvaluation import evaluate_presets
from globals import SAMPLE_RATE, DURATION, PROCESSORS
from IPython.display import Audio

In [3]:
def render_presets(presets):
    synth = Synth(
        sample_rate=SAMPLE_RATE,
        duration=DURATION,
        presets=presets
    )
    return synth.process_audio()

In [ ]:
if __name__ == '__main__':
    target_mfcc = evaluate_presets(load_parameters_file('target.json'))
    # init_solution_mask = np.squeeze(from_preset_to_matrix(normalize_preset(load_parameters_file('init.json'))))

    optimizable_params = PARAM_NAMES

    x0 = np.random.rand(len(optimizable_params))
    sigma0 = 0.5

    # Crear estrategia
    es = cma.CMAEvolutionStrategy(x0, sigma0, {
        'popsize': 50, 
        'maxiter': 1000, 
        'bounds': [0, 1]
    })

    gen = 1

    with Pool(PROCESSORS) as pool:
        while not es.stop():
            # Generar población
            solutions = np.array(es.ask(), dtype=np.float32)  # devuelve una lista de individuos

            # solutions_mask = np.tile(init_solution_mask, (solutions.shape[0], 1)) # Para logica de optimizacion por partes
            # for i, param in enumerate(optimizable_params):
            #     idx = PARAM_NAMES.index(param)
            #     solutions_mask[:,idx] = solutions[:,i]

            solutions_splitted = np.split(solutions, PROCESSORS)
            presets_splitted = [denormalize_preset(from_matrix_to_preset(chunk)) for chunk in solutions_splitted]
            
            solutions_evaluated = pool.map(evaluate_presets, presets_splitted)
            solutions_evaluated = np.concatenate(solutions_evaluated)
            fitnesses = MSE(solutions_evaluated, target_mfcc)

            best_idx = np.argmin(fitnesses)          # índice del mejor fitness
            best_solution = solutions[best_idx]      # solución correspondiente
            best_fitness = fitnesses[best_idx]       # fitness correspondiente

            print("Gen", gen, "Mejor fitness:", best_fitness)

            es.tell(solutions, fitnesses)  # pasar fitness al algoritmo

            gen += 1

    # Mejor solución
    best_solution = es.result.xbest
    print("Mejor individuo:", best_solution)

c:\Users\bryan\Documents\GitHub\Synth\Utils.py:117: RuntimeWarning: divide by zero encountered in log10
  log_f = np.log10(f)        # logaritmo de la frecuencia
c:\Users\bryan\Documents\GitHub\Synth\venv\Lib\site-packages\cma\evolution_strategy.py:1531: UserWarning: Sampling standard deviation i=0 (and 41 others) at iteration 0 multiplied by 0.6666666666666666 to stds[0]=0.3333333333333333
  warnings.warn("Sampling standard deviation i={0}{4} at iteration {1}"


(25_w,50)-aCMA-ES (mu_w=14.0,w_1=14%) in dimension 42 (seed=445142, Mon Jan 12 19:22:28 2026)
Gen 1 Mejor fitness: 0.039852876
Gen 2 Mejor fitness: 0.03209859
Gen 3 Mejor fitness: 0.028867727
Gen 4 Mejor fitness: 0.033322994
Gen 5 Mejor fitness: 0.035871334
Gen 6 Mejor fitness: 0.026487874
Gen 7 Mejor fitness: 0.025287766
Gen 8 Mejor fitness: 0.02826879
Gen 9 Mejor fitness: 0.02655828
Gen 10 Mejor fitness: 0.018280465
Gen 11 Mejor fitness: 0.02528078
Gen 12 Mejor fitness: 0.021256153
Gen 13 Mejor fitness: 0.02478702
Gen 14 Mejor fitness: 0.02398211
Gen 15 Mejor fitness: 0.024602314
Gen 16 Mejor fitness: 0.030719219
Gen 17 Mejor fitness: 0.0208615
Gen 18 Mejor fitness: 0.025596675
Gen 19 Mejor fitness: 0.023678321
Gen 20 Mejor fitness: 0.028325068
Gen 21 Mejor fitness: 0.028253892
Gen 22 Mejor fitness: 0.022198495
Gen 23 Mejor fitness: 0.025940739
Gen 24 Mejor fitness: 0.0259651
Gen 25 Mejor fitness: 0.023639793
Gen 26 Mejor fitness: 0.026032895
Gen 27 Mejor fitness: 0.022911899
Gen 28 

In [ ]:
# solution = init_solution_mask
# for i, param in enumerate(optimizable_params):
#     idx = PARAM_NAMES.index(param)
#     solution[idx] = best_solution[i]

solution = np.expand_dims(best_solution, axis=0)
error = MSE(evaluate_presets(denormalize_preset(from_matrix_to_preset(solution))), target_mfcc)
print('Error', error)

target_audio = render_presets(load_parameters_file('target.json'))
predicted_audio = render_presets(denormalize_preset(from_matrix_to_preset(solution)))

[5.75272948e-04 9.99226809e-01 4.68628168e-01 8.75825156e-03
 5.53209126e-01 3.01718898e-02 2.49738753e-01 1.06996521e-02
 9.76350427e-01 3.44553232e-01 9.98234212e-01 9.76972401e-01
 9.21510279e-01 1.62968681e-05 1.12310247e-02 9.58932579e-01
 6.18909359e-01 9.88987386e-01 2.84557007e-02 8.86948347e-01
 2.76835356e-02 3.56029749e-01 5.21048903e-01 9.95127380e-01
 1.53981164e-01 3.37750018e-01 2.99847405e-02 4.14251722e-03
 6.04351610e-03 9.95417178e-01 4.25447285e-01 9.54481125e-01
 7.62040734e-01 1.29634544e-01 9.97301221e-01 1.01951082e-04
 9.98624146e-01 1.19282736e-03 1.51019782e-01 8.98827553e-01
 1.93920149e-03 3.91726801e-03]
Error [3.367579e-05]


In [6]:
Audio(target_audio[0], rate=SAMPLE_RATE)

In [7]:
Audio(predicted_audio[0], rate=SAMPLE_RATE)

In [8]:
denormalized_predicted_preset = denormalize_preset(from_matrix_to_preset(solution))
pretty_print(denormalized_predicted_preset)

{
  lfo1_rate:   [
    0.2537547905743845
  ]
  lfo1_shape:   [
    3.8179244995117188
  ]
  lfo2_rate:   [
    3.277323573320018
  ]
  lfo2_shape:   [
    0.5185381770133972
  ]
  osc1_shape:   [
    0.00230109179392457
  ]
  osc1_phase:   [
    0.9992268085479736
  ]
  osc1_volume:   [
    0.4686281681060791
  ]
  osc1_freq:   [
    21.265515391725614
  ]
  osc1_vdepth:   [
    0.553209125995636
  ]
  osc1_pdepth:   [
    0.030171889811754227
  ]
  osc2_shape:   [
    0.9989550113677979
  ]
  osc2_phase:   [
    0.010699652135372162
  ]
  osc2_volume:   [
    0.9763504266738892
  ]
  osc2_freq:   [
    223.507126545101
  ]
  osc2_vdepth:   [
    0.998234212398529
  ]
  osc2_pdepth:   [
    0.9769724011421204
  ]
  osc3_shape:   [
    3.6860411167144775
  ]
  osc3_phase:   [
    1.6296868125209585e-05
  ]
  osc3_volume:   [
    0.011231024749577045
  ]
  osc3_freq:   [
    16537.351208864802
  ]
  osc3_vdepth:   [
    0.6189093589782715
  ]
  osc3_pdepth:   [
    0.988987386226654
  ]

In [9]:
np.diagonal(es.C)

array([8.00587848e-01, 1.84457406e+00, 1.62210529e+00, 4.22844189e-01,
       1.18156650e+00, 1.87536905e+00, 7.03271922e-03, 1.49547086e+00,
       3.07081452e-01, 2.63063196e-04, 1.09723162e+00, 4.66300329e-02,
       1.77484693e+00, 9.30862857e-01, 1.86569931e-01, 6.60672441e-01,
       1.35142733e+00, 1.43246886e+00, 4.62325938e-01, 1.64004759e+00,
       5.89712098e-01, 1.04664778e-02, 1.66138477e+00, 1.11028981e+00,
       9.59475607e-01, 6.76271664e-02, 2.37281130e-01, 1.16144927e-01,
       7.58523572e-01, 1.19434926e+00, 4.59882756e-02, 1.87692640e-01,
       9.11494984e-01, 5.02541385e-01, 1.07923267e+00, 3.78335265e-01,
       4.76713201e-01, 4.19306786e-01, 1.52573369e+00, 1.77614937e+00,
       9.69312019e-01, 1.17535343e+00])